In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom')

In [ ]:
!pip install tensorboardX
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitli

In [ ]:
import os
import time
import torch.multiprocessing
from copy import deepcopy
from tensorboardX import SummaryWriter
from tqdm import tqdm
import shutil

from parameters import Parameters
from train.trainer import Trainer
from deepfake_datasets.datasets import get_dataloader
from train.early_stopping import EarlyStopping
from train.validate import validate_detection, validate_fully_supervised_localization
from utils.utils import compute_mean_iou, compute_mean_ap, compute_mean_f1, compute_mean_acc_detection, compute_mean_ap_detection, compute_batch_iou, compute_batch_localization_f1, compute_batch_ap, compute_accuracy_detection, compute_average_precision_detection


In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
new_dataset_name = 'ldm'

zip_path = f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets_zip/{new_dataset_name}.zip'
new_content_root_path = f'/content/datasets/dolos_data/celebahq/fake/'

os.makedirs(new_content_root_path, exist_ok=True)

# move the dataset from drive to /content (SSD) for better performance in I/O
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_content_root_path)

In [ ]:
# set parameters
params = Parameters()

# set experiment name
params.experiment_name = 'training_inital'

# set backbone
params.arch = 'CLIP:ViT-L/14'

# create output dirs
params.create_output_dirs()

# uncomment lines below to set new datasets
# new_drive_root_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets/dolos_data/celebahq/'
# new_root_path = '/content/dataset'
# new_root_path = '..\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'
# new_root_path = 'D:\\Python\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'


# move the dataset from drive to /content for better performance in I/O
# shutil.copytree(new_drive_root_path, new_root_path, dirs_exist_ok=True)

new_root_path = '/content/datasets/dolos_data/celebahq/'
new_dataset_name = 'ldm'

params.update_dolos_data_paths(new_root_path, new_dataset_name)

# fix the backbone - to train only the decoder
params.fix_backbone = True

# set the feature layer and the decoder type
params.feature_layer = 'layer20'
params.decoder_type = 'conv-20'

# set the batch size and num threads
params.batch_size = 64
params.num_threads = 8

# set the number of epochs
params.num_iter = 30

# set loss type
params.loss_type = 'bce'

for key, value in params.__dict__.items():
    print(f"{key}: {value}")

experiment_name: training_inital
task_type: fully_supervised_localization
data_label: train
arch: CLIP:ViT-L/14
fix_backbone: True
weight_decay: 0.0
batch_size: 64
num_threads: 8
init_type: normal
init_gain: 0.02
train_dataset: ldm
decoder_type: conv-20
feature_layer: layer20
early_stop_epochs: 5
optim: adam
beta1: 0.9
lr: 0.001
show_loss_freq: 50
num_iter: 30
data_root_path: /content/datasets/dolos_data/celebahq
train_fake_path: /content/datasets/dolos_data/celebahq/fake/ldm/images/train
valid_fake_path: /content/datasets/dolos_data/celebahq/fake/ldm/images/valid
test_fake_path: /content/datasets/dolos_data/celebahq/fake/ldm/images/test
train_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/ldm/masks/train
valid_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/ldm/masks/valid
test_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/ldm/masks/test
train_real_path: /content/datasets/dolos_data/celebahq/real/train
valid_real_path: /co

In [ ]:
# get the model, along with its trainer
model_trainer = Trainer(params)

# state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/train_all_4/interrupted/best_localization_model_vit_conv20_iou_54.5470_epoch_22.pth', map_location='cpu')
# model_trainer.model.load_state_dict(state_dict['model'], strict=False)
# model_trainer.model = model_trainer.model.cuda()

100%|███████████████████████████████████████| 890M/890M [00:18<00:00, 50.8MiB/s]


In [ ]:
# get the dataloaders
train_loader = get_dataloader(params)

val_params = deepcopy(params)
val_params.data_label = 'valid'
val_loader = get_dataloader(val_params)

In [ ]:
# use summary writers to log training and validation metrics
train_writer = SummaryWriter(os.path.join(params.save_dir_results, 'train'))
val_writer = SummaryWriter(os.path.join(params.save_dir_results, 'valid'))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# start the training loop
# model_trainer.compute_pos_weight_per_dataset(train_loader)
# print('Pos weight for fake pixels', model_trainer.pos_weights)
early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.001)
best_metric = 0.0 # iou for localization, ap for detection
print('Length of training set:', len(train_loader.dataset))
print('Length of validation set:', len(val_loader.dataset))
start_time = time.time()
for epoch in tqdm(range(params.num_iter)):
    print('Epoch:', epoch)

    epoch_loss = 0
    for data in train_loader:
        model_trainer.total_steps += 1

        model_trainer.set_input(data)
        model_trainer.optimize_parameters()

        if model_trainer.total_steps % params.show_loss_freq == 0:
            epoch_loss += model_trainer.loss.item()
            print(f'Train Loss: {model_trainer.loss.item():.4f} at step {model_trainer.total_steps} \t Iter time: {(time.time() - start_time) / model_trainer.total_steps:.2f}')
            train_writer.add_scalar('loss', model_trainer.loss.item(), model_trainer.total_steps)


    epoch_loss /= len(train_loader.dataset)
    print(f'Average Train Loss: {epoch_loss:.4f}')

    # compute training metrics
    if params.task_type == 'fully_supervised_localization':
        compute_mean_iou(model_trainer.ious, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ious = []

        compute_mean_f1(model_trainer.f1_best, model_trainer.f1_fixed, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.f1_best = []
        model_trainer.f1_fixed = []

        compute_mean_ap(model_trainer.ap, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ap = []

    elif params.task_type == 'detection':
        model_trainer.format_output_detection()

        compute_mean_acc_detection(model_trainer.logits, model_trainer.labels, verbose=True, extra_text=f'Train epoch {epoch} ')
        compute_mean_ap_detection(model_trainer.logits, model_trainer.labels, verbose=True, extra_text=f'Train epoch {epoch} ')

        model_trainer.logits = []
        model_trainer.labels = []


    # validate the model
    print('Validation:')
    if params.task_type == 'fully_supervised_localization':
        ious, f1_best, f1_fixed, mean_ap, _ = validate_fully_supervised_localization(model_trainer.model, val_loader, params.train_dataset)

        # compute metrics
        mean_iou = compute_mean_iou(ious, verbose=True, extra_text=f'Validation at epoch {epoch} ')
        val_writer.add_scalar('iou', mean_iou, model_trainer.total_steps)

        mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=f'Validation at epoch {epoch} ')
        val_writer.add_scalar('f1_best', mean_f1_best, model_trainer.total_steps)
        val_writer.add_scalar('f1_fixed', mean_f1_fixed, model_trainer.total_steps)

        mean_ap = compute_mean_ap(mean_ap, verbose=True, extra_text=f'Validation at epoch {epoch} ')
        val_writer.add_scalar('mean_ap', mean_ap, model_trainer.total_steps)

        # save the model if the mean iou is improved
        if mean_iou > best_metric:
            best_metric = mean_iou
            model_trainer.save_model(f'best_localization_model_iou_{mean_iou:.4f}.pth')
            print(f'Best model saved at epoch {epoch}!')

        # check for early stopping
        early_stopping(mean_iou)

    elif params.task_type == 'detection':
        ap, acc, _, _, _ = validate_detection(model_trainer.model, val_loader, params.train_dataset)

        print(f'Validation at epoch {epoch} - AP: {ap:.4f}, Acc: {acc:.4f}')
        val_writer.add_scalar('ap', ap, model_trainer.total_steps)
        val_writer.add_scalar('acc', acc, model_trainer.total_steps)

        # save the model if the mean ap is improved
        if ap > best_metric:
            best_metric = ap
            model_trainer.save_model(f'best_detection_model_ap_{ap:.4f}.pth')
            print(f'Best model saved at epoch {epoch}!')

        # check for early stopping
        early_stopping(ap)

    # check if early stopping is triggered
    if early_stopping.early_stop:
        print("Early stopping triggered")
        continue_training = model_trainer.adjust_learning_rate()
        if continue_training:
            print("Continuing training with a learning rate reduced by a factor of 10")
            early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.002) # adjust the delta only once, otherwise stop completely
        else:
            print(f"Early stopping training at epoch {epoch}")
            break
    print()
    torch.cuda.empty_cache()

Length of training set: 9000
Length of validation set: 900


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0
Train Loss: 0.3378 at step 50 	 Iter time: 2.89
Train Loss: 0.3830 at step 100 	 Iter time: 2.86
Average Train Loss: 0.0001
Train epoch 0 Mean IOU: 0.08
Train epoch 0 Mean F1 best: 0.5874
Train epoch 0 Mean F1 fixed: 0.1041
Train epoch 0 Mean AP: 0.4629
Validation:
Length of dataset:  900



100%|██████████| 15/15 [00:32<00:00,  2.18s/it]


Validation at epoch 0 Mean IOU: 0.0
Validation at epoch 0 Mean F1 best: 0.6257
Validation at epoch 0 Mean F1 fixed: 0.0989
Validation at epoch 0 Mean AP: 0.5122


...

Validation at epoch 27 Mean IOU: 48.39
Validation at epoch 27 Mean F1 best: 0.6739
Validation at epoch 27 Mean F1 fixed: 0.5694
Validation at epoch 27 Mean AP: 0.6761

Epoch: 28
Train Loss: 0.0863 at step 3950 	 Iter time: 3.06
Train Loss: 0.0666 at step 4000 	 Iter time: 3.06
Train Loss: 0.0759 at step 4050 	 Iter time: 3.06
Average Train Loss: 0.0000
Train epoch 28 Mean IOU: 80.07
Train epoch 28 Mean F1 best: 0.9443
Train epoch 28 Mean F1 fixed: 0.916
Train epoch 28 Mean AP: 0.928
Validation:
Length of dataset:  900



 97%|█████████▋| 29/30 [3:28:42<07:09, 429.92s/it]

Validation at epoch 28 Mean IOU: 48.11
Validation at epoch 28 Mean F1 best: 0.6769
Validation at epoch 28 Mean F1 fixed: 0.5674
Validation at epoch 28 Mean AP: 0.68
Early Stopping counter: 1 out of 5

Epoch: 29
Train Loss: 0.0665 at step 4100 	 Iter time: 3.06
Train Loss: 0.0796 at step 4150 	 Iter time: 3.06
Train Loss: 0.0793 at step 4200 	 Iter time: 3.06
Average Train Loss: 0.0000
Train epoch 29 Mean IOU: 80.1
Train epoch 29 Mean F1 best: 0.9472
Train epoch 29 Mean F1 fixed: 0.9183
Train epoch 29 Mean AP: 0.9285
Validation:
Length of dataset:  900



100%|██████████| 30/30 [3:35:51<00:00, 431.72s/it]

Validation at epoch 29 Mean IOU: 48.25
Validation at epoch 29 Mean F1 best: 0.6731
Validation at epoch 29 Mean F1 fixed: 0.5728
Validation at epoch 29 Mean AP: 0.6735
Early Stopping counter: 2 out of 5



In [ ]:
model_dir = '/content/experiments/training_inital/models/'
model_files = sorted(
    [f for f in os.listdir(model_dir) if f.endswith('.pth')]
)

best_model = model_files[-1]
best_model_path = os.path.join(model_dir, best_model)

In [ ]:
shutil.copy(best_model_path, f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/{os.path.basename(best_model_path)}')

'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/initial_training/best_localization_model_iou_48.4968.pth'

In [ ]:
from google.colab import files

files.download(best_model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>